In [10]:
import pandas as pd
import numpy as np

In [11]:
teams = pd.read_csv('data/mens/MTeams.csv')

In [12]:
teams_spellings = pd.read_csv('data/mens/MTeamSpellings.csv', encoding='latin-1')

In [13]:
def teamname_to_teamid(teamname):
        
    mappings = {'Milwaukee': 'WI Milwaukee',
                'WI Green Bay': 'Green Bay',
                'Boston University': 'Boston Univ',
               'Southern': 'Southern Univ',
               'Cal St. Fullerton': 'CS Fullerton',
               'Cal St. Northridge': 'CS Northridge',
               'Cal St. Bakersfield': 'CS Bakersfield',
               'Maryland Eastern Shore': 'MD E Shore',
               'Mississippi Valley St.': 'MS Valley St',
               'Illinois Chicago': 'IL Chicago',
               'Arkansas Pine Bluff': 'Ark Pine Bluff',
                'Arkansas Little Rock': 'Ark Little Rock',
               'Southeast Missouri St.': 'SE Missouri St',
               'Louisiana Monroe': 'ULM',
               'Bethune Cookman': 'Bethune-Cookman',
               'Texas A&M Corpus Chris': 'TAM C. Christi',
                'Tennessee Martin': 'TN Martin',
                'UT Rio Grande Valley': 'UTRGV',
                'NJ Inst of Technology': 'NJIT',
                'Virginia Military Inst': 'VMI',
                'Missouri Kansas City': 'Missouri KC',
                'Louisiana St.': 'LSU',
                'Florida A&M;': 'Florida A&M',
                'Wisconsin Milwaukee': 'WI Milwaukee',
                'Nevada Las Vegas': 'UNLV',
                'Alabama A&M;': 'Alabama A&M',
                'Texas A&M; Corpus Chris': 'TAM C. Christi',
                'North Carolina A&T;':'NC A&T',
                'IUPU Fort Wayne': 'PFW',
                'Wisconsin Green Bay': 'WI Green Bay',
                'Texas El Paso': 'UTEP',
                'Texas A&M;' : 'Texas A&M',
                'Prairie View A&M;': 'Prairie View'
               }
    try:
        teamname = mappings[teamname]
    except KeyError:
        pass
        
    try:
        teamid = teams[teams['TeamName'] == teamname].iloc[0]['TeamID']
        return teamid
    except (KeyError, IndexError):
        pass

    try:
        teamid = teams[teams['TeamName'] == teamname.replace('.','')].iloc[0]['TeamID']
        return teamid
    except (KeyError, IndexError):
        pass

    try:
        teamid = teams[teams['TeamName'] == teamname.replace("'","")].iloc[0]['TeamID']
        return teamid
    except (KeyError, IndexError):
        pass
    
    if teamname.endswith('St.'):
        try:
            teamid = teams[teams['TeamName'] == teamname.rstrip('.')].iloc[0]['TeamID']
            return teamid
        except (KeyError, IndexError):
            pass

    try:
        teamid = teams_spellings[teams_spellings['TeamNameSpelling'] == teamname.lower()].iloc[0]['TeamID']
        return teamid
    except (KeyError, IndexError):
        pass
    
    try:
        teamid = teams_spellings[teams_spellings['TeamNameSpelling'] == teamname.lower().replace('.','')].iloc[0]['TeamID']
        return teamid
    except (KeyError, IndexError):
        return np.nan

In [14]:
PiT_urls = {2011: 'https://web.archive.org/web/20110228211802/https://kenpom.com/',
           2012: 'https://web.archive.org/web/20120301065140/https://kenpom.com/',
           2013: 'https://web.archive.org/web/20130301122807/https://kenpom.com/',
           2014: 'https://web.archive.org/web/20140301071232/https://kenpom.com/',
           2015: 'https://web.archive.org/web/20150228131430/https://kenpom.com/',
           2016: 'https://web.archive.org/web/20160301131933/http://kenpom.com/',
           2017: 'https://web.archive.org/web/20170301072239/https://kenpom.com/',
           2018: 'https://web.archive.org/web/20180301162901/https://kenpom.com/',
           2019: 'https://web.archive.org/web/20190301160638/https://kenpom.com/',
           2020:'https://kenpom.com/index.php'}

In [15]:
def get_and_match_kenpoms(year=2020):
    url = PiT_urls[year]
    dfs = pd.read_html(url)
    if len(dfs[0].columns) > 20:
        df = dfs[0]
    elif len(dfs[1].columns) > 20:
        df = dfs[1]
    elif len(dfs[-1].columns) > 20:
        df = dfs[-1]
    real_cols = [df.columns[i][-1].replace('.1','') for i in range(len(df.columns))]
    df.columns = real_cols
    df = df.loc[:,~df.columns.duplicated()]
    df = df[~pd.isna(df['Team'])]
    df['Team'] = df['Team'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
    df['Team'] = df['Team'].apply(lambda x: x.rstrip(' '))
    df['TeamID'] = df['Team'].apply(lambda x: teamname_to_teamid(x))
    df = df[~pd.isna(df['TeamID'])]
    if 'AdjEM' not in df.columns:
        df['AdjEM'] = df['AdjO'].apply(lambda x: float(x)) - df['AdjD'].apply(lambda x: float(x))
        del df['Pyth']
    return df

In [16]:
all_dfs = {}
for year in range(2011, 2021):
    df = get_and_match_kenpoms(year=year)
    all_dfs[year] = df
    print(year, df.shape)
    df.to_csv(f'data/mens/PiT_kenpoms_{year}.csv')

2011 (342, 12)
2012 (342, 12)
2013 (344, 12)
2014 (349, 12)
2015 (349, 12)
2016 (350, 12)
2017 (350, 12)
2018 (350, 12)
2019 (353, 12)
2020 (352, 12)
